# The Tensor objects

In [ ]:
import pixelprism.math as pm
import numpy as np

## Creation

A `Tensor` is PixelPrism's friendly wrapper around NumPy arrays. It keeps the data, the shape, and the dtype in one tidy place, while staying easy to construct in a bunch of ways. Let's tour every creation path in `pixelprism/math/tensor.py`.

### 1) Direct constructor: scalars, vectors, matrices

The `Tensor` constructor accepts numbers, lists, nested lists, and NumPy arrays. Shapes are inferred automatically.

In [ ]:
scalar = pm.Tensor(3)
scalar


In [ ]:
vector = pm.Tensor([1, 2, 3])
matrix = pm.Tensor([[1, 2], [3, 4]])
vector, matrix


A single number creates a rank-0 tensor (a scalar). A 1D list becomes a vector, and nested lists become matrices or higher-rank tensors.

### 2) From NumPy arrays

If you already have NumPy data, pass it directly. NumPy scalars also work.

In [ ]:
np_vec = np.array([10, 20, 30], dtype=np.int32)
np_scalar = np.array(2.5)

from_np_vec = pm.Tensor(np_vec)
from_np_scalar = pm.Tensor(np_scalar)
from_np_vec, from_np_scalar


### 3) Static helpers for clarity

These are thin wrappers around the constructor, but they are explicit and readable in tutorials or pipelines.

In [ ]:
explicit_np = pm.Tensor.from_numpy(np.arange(6).reshape(2, 3))
print(explicit_np)


In [ ]:
explicit_list = pm.Tensor.from_list([1, 2, 3], dtype=pm.Z)
explicit_list


`from_list` requires a `DType` so there is no ambiguity about integers vs floats.

### 4) Zeros factory

`Tensor.zeros` creates a tensor filled with zeros. The shape can be an int, a tuple, a list, or a `TensorShape`.

In [ ]:
zeros_tuple = pm.t_zeros((2, 3))
zeros_int = pm.t_zeros(4)
zeros_shape = pm.t_zeros(pm.ts_matrix(2, 2))
zeros_tuple, zeros_int, zeros_shape


### 5) Dtype and mutability knobs

The constructor accepts `dtype` and `mutable`. `dtype` can be a `pm.DType`, a NumPy dtype, or a Python type (`int`, `float`, `bool`, `complex`). `mutable=False` makes the tensor read-only for in-place ops.

In [ ]:
float_tensor = pm.Tensor([1, 2, 3], dtype=pm.DType.R)
bool_tensor = pm.Tensor([1, 0, 1], dtype=bool)
immutable = pm.Tensor([1, 2, 3], mutable=False)
float_tensor, bool_tensor, immutable


## Algebra

Tensors are made for algebra. Every operation below is **elementwise**: each number talks only to its matching partner. This mirrors how we build models in machine learning: data flows through layers as a chain of simple, reliable transforms.

### 1) The four basics: +, -, *, /

Think of these as the arithmetic gears of a neural network: add a bias, scale activations, normalize, or compute residuals.

In [ ]:
x = pm.Tensor([1.0, 2.0, 3.0])
y = pm.Tensor([10.0, 20.0, 30.0])

x_plus_y = x + y
x_minus_y = x - y
x_times_y = x * y
x_div_y = x / y

x_plus_y, x_minus_y, x_times_y, x_div_y


You can also mix tensors with scalars. Broadcasting applies the scalar across every element.

In [ ]:
shift = x + 0.5
scale = x * 2
normalize = x / 10
shift, scale, normalize


You can also mix tensors with Numpy array. Broadcasting applies the scalar across every element.

In [ ]:
shift = x + np.array(0.5)
shift_vector = x + np.array([0.5, 0.5, 0.5])
scale = x * np.array(2)
normalize = x / np.array([10, 10, 10])
shift, scale, normalize

### 2) Power moves: `**`, `pow`, and friends

In ML, exponents show up in loss functions (squared error), normalization, and feature engineering.

In [ ]:
squared_op = x ** 2
squared_method = x.pow(2)

cube = x ** 3
fractional = x ** 0.5

squared_op, squared_method, cube, fractional


`**` and `pow` are equivalent; choose the style that reads best in your notebook.

### 3) Convenience helpers: `square()` and `cbrt()`

These are readable shortcuts for common transforms. `square()` is the workhorse of least-squares losses, while `cbrt()` is a gentler nonlinearity.

In [ ]:
square = x.square()
root3 = x.cbrt()

square, root3


### 4) Advanced unary operations

These functions show up everywhere in mathematics and machine learning: exponentials and logs drive probability and optimization, while absolute values underpin L1 regularization and robust losses.

We keep inputs in safe domains (e.g., positive values for logs) to avoid undefined results.

In [ ]:
u = pm.Tensor([0.1, 0.5, 1.0])

recip = u.reciprocal()  # 1/x
exp_u = u.exp()
exp2_u = u.exp2()
expm1_u = u.expm1()

recip, exp_u, exp2_u, expm1_u


In [ ]:
v = pm.Tensor([1.0, 2.0, 10.0])

log_v = v.log()
log2_v = v.log2()
log10_v = v.log10()
log1p_v = v.log1p()

log_v, log2_v, log10_v, log1p_v


`log1p` and `expm1` are numerically stable for small values and often used in loss functions and probability computations.

In [ ]:
w = pm.Tensor([-3.0, -1.5, 2.0])

abs_w = w.absolute()
abs_alias = w.abs()

abs_w, abs_alias


`absolute()` and `abs()` are equivalent; `abs()` mirrors NumPy's naming.

### 5) Comparisons and boolean masks

Comparisons are the gatekeepers of math and ML: they decide which elements pass a threshold, which errors are large, or which predictions are correct. There are two flavors in PixelPrism:

Python operators like `==` and `<` return a single `bool`. In this implementation `==`, `!=`, `<=`, and `>=` require *all* elements to satisfy the comparison, while `<` and `>` return `True` if *any* element satisfies it.

Tensor methods like `equal()` and `less()` return a **Tensor of booleans**, perfect for masks.

In [ ]:
a = pm.Tensor([1.0, 2.0, 3.0])
b = pm.Tensor([1.0, 0.0, 3.0])

op_eq = a == b
op_ne = a != b
op_lt = a < b
op_le = a <= b
op_gt = a > b
op_ge = a >= b

op_eq, op_ne, op_lt, op_le, op_gt, op_ge


The operator comparisons return `bool` values (useful for quick checks), while method comparisons return elementwise masks.

In [ ]:
mask_eq = a.equal(b)
mask_ne = a.not_equal(b)
mask_lt = a.less(b)
mask_le = a.less_equal(b)
mask_gt = a.greater(b)
mask_ge = a.greater_equal(b)

mask_eq, mask_ne, mask_lt, mask_le, mask_gt, mask_ge


These masks are the building blocks of ML logic: use them to select values, build losses, or filter data.

### 6) Trigonometry and hyperbolic friends

Trigonometry powers rotations, waves, and periodic signals. In ML it pops up in positional encodings, phase features, and Fourier-like representations. PixelPrism follows NumPy: **angles are in radians** by default.

In [ ]:
angles = pm.Tensor([0.0, np.pi / 6, np.pi / 2, np.pi])

sin_a = angles.sin()
cos_a = angles.cos()
tan_a = angles.tan()

sin_a, cos_a, tan_a


Inverse trig functions need values in their valid domains (e.g., `arcsin`/`arccos` for inputs in [-1, 1]).

In [ ]:
x_unit = pm.Tensor([-1.0, -0.5, 0.0, 0.5, 1.0])

asin_x = x_unit.arcsin()
acos_x = x_unit.arccos()
atan_x = x_unit.arctan()

asin_x, acos_x, atan_x


`arctan2` is the angle of a 2D vector (y, x). It's great for turning coordinates into directions.

In [ ]:
y = pm.Tensor([0.0, 1.0, 1.0])
x = pm.Tensor([1.0, 1.0, -1.0])

angles_2d = y.arctan2(x)
angles_2d


Hyperbolic functions appear in activation functions (think `tanh`) and in smooth feature transforms.

In [ ]:
h = pm.Tensor([-2.0, -1.0, 0.0, 1.0, 2.0])

sinh_h = h.sinh()
cosh_h = h.cosh()
tanh_h = h.tanh()

sinh_h, cosh_h, tanh_h


Inverse hyperbolic functions have their own valid domains; keep inputs in safe ranges when exploring.

In [ ]:
h_small = pm.Tensor([-0.9, 0.0, 0.9])

asinh_h = h_small.arcsinh()
atanh_h = h_small.arctanh()

asinh_h, atanh_h


In [ ]:
h_pos = pm.Tensor([1.0, 2.0, 5.0])

acosh_h = h_pos.arccosh()
acosh_h


Need degrees? Convert with `deg2rad()` or `rad2deg()` before or after trig calls.

In [ ]:
deg = pm.Tensor([0.0, 30.0, 90.0, 180.0])

rad = deg.deg2rad()
back_to_deg = rad.rad2deg()

rad, back_to_deg


## Rendering Tensors with Latex

In [ ]:
from pixelprism.math.render.latex import to_latex
to_latex(apbpc)

In [ ]:
from pixelprism.math.render import render_latex
latex_image = render_latex(apbpc, output_path="latex_image.svg")
latex_image